### Setup

In [1]:
import os, sys, importlib, time
from dotenv import load_dotenv

# Get directories first to find .env file
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
env_path = os.path.join(parent_dir, '.env')

# Load .env file
load_dotenv(env_path, override=True)

# Get timezone from .env with default fallback
timezone = os.getenv('TZ', 'America/New_York')
os.environ["TZ"] = timezone
time.tzset()

# Get additional directories
rag_dir = os.path.join(parent_dir, 'RAG')
print(f"Current directory: {current_dir}")
print(f"Parent directory: {parent_dir}")
print(f"Parent dir exists: {os.path.exists(parent_dir)}")
print(f"RAG dir exists: {os.path.exists(rag_dir)}")
print(f"KnowledgeGraph exists: {os.path.exists(os.path.join(parent_dir, 'KnowledgeGraph'))}")

# Add paths to sys.path BEFORE importing modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
if rag_dir not in sys.path:
    sys.path.insert(0, rag_dir)
    
# Import modules after paths are set up
from KnowledgeGraph.config import load_neo4j_graph
import RAG.VectorRAG as VectorRAG
import RAG.GraphRAG as GraphRAG
import RAG.batch_query as batch_query

# make sure AB Testing is on sys.path
test_dir = os.path.join(parent_dir, "AB Testing")
if test_dir not in sys.path:
    sys.path.insert(0, test_dir)

# import/reload mark_answers, then get the function
import mark_answers as mark_answers_module
importlib.reload(mark_answers_module)
mark_answers = mark_answers_module.mark_answers

# Reload other modules to pick up changes
importlib.reload(VectorRAG)
importlib.reload(GraphRAG)
importlib.reload(batch_query)

from RAG.GraphRAG import query_graph_rag
graph, openAI_api, openAI_endpoint, openAI_model = load_neo4j_graph()

Current directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/Main Functions
Parent directory: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3
Parent dir exists: True
RAG dir exists: True
KnowledgeGraph exists: True


/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j/_sync/driver.py:547: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(


### GraphRag Singluar Query

In [4]:
catalog_question = "For tender 2021/2020/06, what are the closing date, closing time, bid validity, tender document fee, whether alternative offers are allowed, and whether the tender is indivisible?"
supplement_question = "What is the company name listed for Tender RFP-2113556?"
failed_question = "Across both the late-April Amiri Diwan tender and Practice A/M/804, are alternative offers allowed?"
failed_question2 = "What is the practice fee for Ministry of Health Practice No. 5DX003?"

GraphRAG.query_graph_rag(failed_question2)

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/retrievers/vector.py:201: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_params = get_search_query(


'75 [1]\n\nReferences\n1. 71-1737-0.'

### GraphRAG Batch Query

In [2]:
qp1 = "AB Testing/questions_long.json"
qp2 = "AB Testing/questions_supplement.json"
qp3 = "AB Testing/questions_short.json"

input_question_file = qp1

batch_query.batch_query_graph_rag(input_question_file = input_question_file, max_workers=12)

/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/retrievers/vector.py:201: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_params = get_search_query(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/retrievers/vector.py:201: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_params = get_search_query(
/home/sepeh/.cache/pypoetry/virtualenvs/graphrag-developer-challenge3-4suqu0LP-py3.12/lib/python3.12/site-packages/neo4j_graphrag/retrievers/vector.py:201: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  search_query, search_para

Processed 71 queries with 12 workers
Results saved to: /mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_205719.json


[{'question': 'What is the time period to submit an objection for company registration changes?',
  'answer': '- Most company registration changes (e.g., amending legal entity, dissolution/liquidation via the Department of Partnerships): 30 days from the announcement’s publication date to file an objection with the General Court (المحكمة الكلية).\n- Conversions of an individual license (sole establishment) via the Companies Department: 15 days from publication to submit a written objection to the Department, accompanied by a debt instrument.'},
 {'question': 'Where should objections to company registration be submitted?',
  'answer': 'To the General Court (Court of First Instance, المحكمة الكلية).'},
 {'question': 'What types of legal entities are mentioned in company registration announcements?',
  'answer': '- One-Person Company (Single Person Company, SPC) — شركة الشخص الواحد\n- Limited Liability Company (LLC) — شركة ذات مسؤولية محدودة\n- General Partnership Company — شركة تضامنية\n

### Mark Answers

In [3]:
if input_question_file == qp1:
    answer_key_path = "AB Testing/answer_key_long.json"
elif input_question_file == qp2:
    answer_key_path = "AB Testing/answer_key_supplement.json" 
elif input_question_file == qp3:
    answer_key_path = "AB Testing/answer_key_short.json"

mark_answers(answer_key_path = answer_key_path, max_workers = 60)


TEST RESULTS

Configuration:
  top_k: 100
  MMR_k: 50
  MMR_LAMBDA: 0.4
  ALWAYS_KEEP_TOP: 10
  neighbor_window: 1

Results:
  Average F1: 52.7% (0.527)

Card-first gate:
  CARD_FIRST_ENABLED: True
  ALLOW_DIRECT_ANSWER: True



{'saved_at': '2025-11-02T20:57:52',
 'expected_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/answer_key_long.json',
 'answers_file': '/mnt/c/Users/sepeh/OneDrive/Documents/Git/GraphRag_Developer_Challenge3/AB Testing/Output Answers/answers_20251102_205719.json',
 'config': {'top_k': 100,
  'MMR_k': 50,
  'MMR_LAMBDA': 0.4,
  'ALWAYS_KEEP_TOP': 10,
  'neighbor_window': 1,
  'CARD_FIRST_ENABLED': True,
  'ALLOW_DIRECT_ANSWER': True},
 'overall': {'evaluated': 71.0,
  'average_f1': 0.5273239436619718,
  'average_f1_percent': 52.73239436619718},
 'questions': [{'question': 'What is the time period to submit an objection for company registration changes?',
   'expected': 'Thirty days from the date of publication in the official gazette [1].\n\nReferences\n1. Kuwait Official Gazette 2025-04-27, Lines 5228-5327.',
   'answer': '- Most company registration changes (e.g., amending legal entity, dissolution/liquidation via the Department of Partnershi